This GAN is trained to generate data with uniform distribution. The Real data (R) used in training is sample set from uniform distribution U(1,5). The Noise data (I) is samplesd from a normal distribution N(10,1.25).

## Load Libs and Functions

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import pickle
from matplotlib import pyplot as plt
import pandas as pd


class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f

    def forward(self, x):
        x = self.map1(x)
        x = self.f(x)
        x = self.map2(x)
        x = self.f(x)
        x = self.map3(x)
        return x

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f

    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return self.f(self.map3(x))

def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]

def get_moments(d):
    mean = torch.mean(d)
    diffs = d - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    final = torch.cat((mean.reshape(1,), std.reshape(1,)))
    return final

## Gaussian and Unifrom Sampler Parameters

In [9]:
#Uniform Distribution Parameter
data_lower = 1
data_upper = 5

#Gaussian Distribution Parameters
data_mean = 10
data_stddev = 1.25

## Real (R) and Noise (I) Data Samplers

In [10]:
# Real Data Sampler - R
# Uniform Distribution with Parameters (a,b) = ()

def get_distribution_sampler(a,b):
    return lambda m, n: (a-b) * torch.rand(m, n) + b

# Noise/Fake Data Sampler - I
# Gaussian Distribution
def get_generator_input_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (n,1)))

## Generative (G) and Discrimative (D) Models Hyper Paratemer 

In [26]:
g_input_size = 1      # Random noise dimension coming into generator, per output vector
g_hidden_size = 5     # Generator complexity
g_output_size = 1     # Size of generated output vector
d_input_size = 500    # Minibatch size - cardinality of distributions
d_hidden_size = 10    # Discriminator complexity
d_output_size = 1     # Single dimension for 'real' vs. 'fake' classification
minibatch_size = d_input_size

d_learning_rate = 1e-3
g_learning_rate = 1e-3
sgd_momentum = 0.9

num_epochs = 5000
print_interval = 100
d_steps = 20
g_steps = 20

dfe, dre, ge = 0, 0, 0
d_real_data, d_fake_data, g_fake_data = None, None, None

(name, preprocess, d_input_func) = ("2 Moments", lambda data: get_moments(data), lambda x: 2)

discriminator_activation_function = torch.sigmoid
generator_activation_function = torch.tanh

d_sampler = get_distribution_sampler(data_lower,data_upper)
gi_sampler = get_generator_input_sampler(data_mean,data_stddev)

G = Generator(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size,
                  f=generator_activation_function)
D = Discriminator(input_size=d_input_func(d_input_size),
                  hidden_size=d_hidden_size,
                  output_size=d_output_size,
                  f=discriminator_activation_function)
criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
d_optimizer = optim.SGD(D.parameters(), lr=d_learning_rate, momentum=sgd_momentum)
g_optimizer = optim.SGD(G.parameters(), lr=g_learning_rate, momentum=sgd_momentum)


## Training G and D Models 

In [27]:
for epoch in range(num_epochs):
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            D.zero_grad()

            #  1A: Train D on real
            d_real_data = Variable(d_sampler(d_input_size, 1))
            d_real_decision = D(preprocess(d_real_data))
            d_real_error = criterion(d_real_decision, Variable(torch.ones([1,1])))  # ones = true
            d_real_error.backward() # compute/store gradients, but don't change params

            #  1B: Train D on fake
            d_gen_input = Variable(gi_sampler(minibatch_size))
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            d_fake_decision = D(preprocess(d_fake_data.t()))
            d_fake_error = criterion(d_fake_decision, Variable(torch.zeros([1,1])))  # zeros = fake
            d_fake_error.backward()
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()

            dre, dfe = extract(d_real_error)[0], extract(d_fake_error)[0]

        for g_index in range(g_steps):
            # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()

            gen_input = Variable(gi_sampler(minibatch_size))
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data.t()))
            g_error = criterion(dg_fake_decision, Variable(torch.ones([1,1])))  # Train G to pretend it's genuine

            g_error.backward()
            g_optimizer.step()  # Only optimizes G's parameters
            ge = extract(g_error)[0]
        
        
        
        if epoch % print_interval == 0:
            print("Epoch %s: D (%s real_err, %s fake_err) G (%s err); Real Dist (%s),  Fake Dist (%s) " %
                  (epoch, dre, dfe, ge, stats(extract(d_real_data)), stats(extract(d_fake_data))))
   
            

torch.save(G.state_dict(),'Generator_Model.pth')
torch.save(D.state_dict(),'Discriminator_Model.pth')

Epoch 0: D (0.5011354684829712 real_err, 0.9274313449859619 fake_err) G (0.5066295862197876 err); Real Dist ([3.006579041481018, 1.1488099434083934]),  Fake Dist ([-0.19709156453609467, 3.3467388593572392e-06]) 
Epoch 100: D (0.6901128888130188 real_err, 0.6882188320159912 fake_err) G (0.6964195966720581 err); Real Dist ([3.003831624984741, 1.1492294472304558]),  Fake Dist ([4.26985502243042, 0.0001671132534811183]) 
Epoch 200: D (0.18668901920318604 real_err, 0.1884099543094635 fake_err) G (1.7607507705688477 err); Real Dist ([2.9981406927108765, 1.147893298305485]),  Fake Dist ([1.441666841506958, 0.00015362305674211532]) 
Epoch 300: D (0.028866177424788475 real_err, 0.03541802242398262 fake_err) G (3.3588197231292725 err); Real Dist ([2.9912586212158203, 1.1495651879431745]),  Fake Dist ([1.8503392934799194, 7.977111234951239e-05]) 
Epoch 400: D (0.016295218840241432 real_err, 0.016464080661535263 fake_err) G (4.115121364593506 err); Real Dist ([3.0013235807418823, 1.152954324563711

Epoch 4000: D (0.693008542060852 real_err, 0.6933802366256714 fake_err) G (0.6929229497909546 err); Real Dist ([3.002164125442505, 1.1493723331918342]),  Fake Dist ([1.2384406328201294, 1.91048434645144]) 
Epoch 4100: D (0.6935232281684875 real_err, 0.6929752826690674 fake_err) G (0.6935425996780396 err); Real Dist ([3.0089781284332275, 1.1493054328284142]),  Fake Dist ([1.4834959506988525, 1.7436713875028333]) 
Epoch 4200: D (0.6929671764373779 real_err, 0.693234920501709 fake_err) G (0.6930090188980103 err); Real Dist ([2.9970507621765137, 1.1528917603349575]),  Fake Dist ([1.5200074911117554, 1.7292059734291623]) 
Epoch 4300: D (0.6933943629264832 real_err, 0.6929514408111572 fake_err) G (0.693332850933075 err); Real Dist ([2.9966161251068115, 1.1517132952912574]),  Fake Dist ([1.7154915928840637, 1.6422024981804864]) 
Epoch 4400: D (0.6930056810379028 real_err, 0.693150520324707 fake_err) G (0.6929960250854492 err); Real Dist ([2.996348738670349, 1.1506876962014214]),  Fake Dist ([

## Load the trained G and D Models

In [ ]:
G = Generator(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size,
                  f=generator_activation_function)
D = Discriminator(input_size=d_input_func(d_input_size),
                  hidden_size=d_hidden_size,
                  output_size=d_output_size,
                  f=discriminator_activation_function)

G.load_state_dict(torch.load('Generator_Model.pth'))
D.load_state_dict(torch.load('Discriminator_Model.pth'))

## Generate Fake Data using G

In [68]:
noise_input = Variable(gi_sampler(minibatch_size))
generated_data = extract(G(noise_input))

## Sample from Real Data (R) Sampler

In [69]:
real_data = extract(Variable(d_sampler(d_input_size, 1)))

## Stats Comparison between Fake and Real Data

In [70]:
stats = {}
stats['Generated'] = [round(np.mean(generated_data),2),round(np.std(generated_data),2)]
stats['Real'] = [round(np.mean(real_data),2),round(np.std(real_data),2)]
stats['Expected'] = [round((data_upper+data_lower)/2,2),round((data_upper-data_lower)/3.464,2)]
stats_df = pd.DataFrame(stats)
stats_df.index = ['Mean','Std Dev']

In [71]:
stats_df

,Generated,Real,Expected
Mean,3.03,2.97,3.00
Std Dev,1.18,1.15,1.15
